## 정적 웹 페이지 크롤링

In [10]:
from bs4 import BeautifulSoup
import urllib.request

result = []

In [11]:
for page in range(1,58):
  Hollys_url = 'https://www.hollys.co.kr/store/korea/korStore.do?pageNo=%d&sido=&gugun=&store=' %page
  html = urllib.request.urlopen(Hollys_url)
  soupHollys = BeautifulSoup(html, 'html.parser')
  tag_tbody = soupHollys.find('tbody')
  for store in tag_tbody.find_all('tr'):
    if len(store) <= 3:
      break
    store_td = store.find_all('td')
    store_name = store_td[1].string
    store_sido = store_td[0].string
    store_address = store_td[3].string
    store_phone = store_td[5].string
    result.append([store_name] + [store_sido] + [store_address] + [store_phone])


In [12]:
len(result)

567

In [13]:
import pandas as pd

hollys_tbl = pd.DataFrame(result, columns = ('store', 'sido-gu','address','phone'))
hollys_tbl.to_csv("hollys.csv", encoding = "cp949", mode = "w", index = True)

## 동적 웹 페이지 크롤링 
Selenium 라이브러리

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [12]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import urllib.request
import pandas as pd
import datetime



CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"    
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)

  
wd.get(CoffeeBean_URL)
time.sleep(1) # 웹페이지 연결할 동안 1초 대기
wd.execute_script('storePop2(1)')
time.sleep(1)
html = wd.page_source
      
soupCB = BeautifulSoup(html, 'html.parser')
store_name_h2 = soupCB.select('div.store_txt > h2')
store_name = store_name_h2[0].string
print(store_name)

학동역 DT점


In [14]:
store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
print(store_info)

[<td>평일 06:30~22:00ㅣ주말/공휴일 07:30~22:00</td>, <td>DT(드라이브 스루) 매장입니다. (주차는 불가)</td>, <td>서울시 강남구 학동로 211 1층  <!--span class="lot">(서울시 강남구 학동로 211 1층)</span--></td>, <td>02-3444-9973</td>]


In [15]:
store_phone = store_info[3].string
store_phone 

'02-3444-9973'

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import urllib.request
import pandas as pd
import datetime

#[CODE 1]
def CoffeeBean_store(result):
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')        # Head-less 설정
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome('chromedriver',options=options)
             
    for i in range(1, 370):  #매장 수 만큼 반복
        wd.get(CoffeeBean_URL)
        time.sleep(1)  #웹페이지 연결할 동안 1초 대기
        try:
            wd.execute_script("storePop2(%d)" %i)
            time.sleep(1) #스크립트 실행 할 동안 1초 대기
            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string
            print(store_name)  #매장 이름 출력하기
            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address_list = list(store_info[2])
            store_address = store_address_list[0]
            store_phone = store_info[3].string
            result.append([store_name]+[store_address]+[store_phone])
        except:
            continue 
    return

#[CODE 0]
def main():
    result = []
    print('CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    CoffeeBean_store(result)  #[CODE 1]
    
    CB_tbl = pd.DataFrame(result, columns=('store', 'address','phone'))
    CB_tbl.to_csv('CoffeeBean.csv', encoding='cp949', mode='w', index=True)

if __name__ == '__main__':
     main()